***Laboratorium 5***

In [ ]:
from dash import Dash, html
app = Dash(__name__)
app.layout = html.Div([
    html.Div("Hello World!")
])

In [ ]:
from dash import dcc, Output, Input, callback
import plotly.express as px

app.layout = html.Div([
    html.Div("Położenie użytkowników"),
    dcc.Slider(id="slider", min=5, max=10, value=5,
               marks={i: str(i) for i in range(5, 11)}),
    dcc.Graph("wykres")
])
@callback(
    Output('wykres', 'figure'),
    Input('slider', 'value')
)
def update_figure(ile_liczb):
    ile_liczb = int(ile_liczb)
    x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    y = [4, 1, 4, 2, 5, 1, 5, 1, 2, 5]
    return px.line(
        x=x[:ile_liczb],
        y=y[:ile_liczb],
        markers=True
    )

In [ ]:
from dash import dash_table
import pandas as pd

# wczytaj plik csv z pogodą (można wspomóc się stroną https://www.convertcsv.com/csv-viewer-editor.htm)
df = pd.read_csv('pogoda.csv')

app.layout.children.append(
    dash_table.DataTable(
        data=df.to_dict('records'), page_size=7
    )
)

In [ ]:
# dodaj menu wyboru kraju (drop down)
kraje = options=df['kraj'].unique()
app.layout.children.append(
    dcc.Dropdown(
        options=kraje,
        value=None,
        id='wybor-kraju',
    )
)

# dodaj menu wyboru dzien/noc (radio buttons)
app.layout.children.append(
    dcc.RadioItems(
        options=['dzien', 'noc'],
        value='dzien',
        id='wybor-dzien-noc'
    )
)

In [ ]:
app.layout.children.append(
    dcc.Graph(
        figure={},
        id='wykres-temperatury'  
    )
)

In [ ]:
@callback(
    Output('wykres-temperatury', 'figure'),
    Input('wybor-kraju', 'value'),
    Input('wybor-dzien-noc', 'value')   
)
def update_temperature_figure(kraj, dzien):
    if dzien == 'dzien':
        dzien = 'temperatura_dzien'
    else:
        dzien = 'temperatura_noc'
    if kraj is None:
        return px.histogram(df, x='kraj', y=dzien, histfunc='avg')
    else:
        df_filter = df[df['kraj'] == kraj]
        return px.bar(df_filter, x='data', y=dzien)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)